Covid Cases Stratified

Author: Zachary Stanke


In [1]:
import os
os.chdir('C:/Users/zakst/Documents/NIH')
os.getcwd()

'C:\\Users\\zakst\\Documents\\NIH'

In [2]:
# Initialization
import numpy as np
from numpy import linalg as la
import pandas as pd
from scipy import optimize
import CovidEM
import CovidCI
import warnings
import math
import sys
from tabulate import tabulate
import csv

import imp
imp.reload(CovidEM) #applies changes made to file

# Read in data
cases = pd.read_csv('CovidStratified/Data/cases.csv', index_col=0)
cases = cases.sort_index(ascending = True)
# to retrieve entries as plain data types: cases.iloc[i,j].values
cases_dict = {}
cases_mat = cases.to_dict("split")["data"] # temp variable to hold onto arrays
for i in range(len(cases)):
    cases_dict[cases.index[i]] = cases_mat[i]
del cases_mat

prem = {
"can" : pd.read_csv('CovidStratified/Data/premCan.csv', index_col=0).to_numpy(),
"chn" : pd.read_csv('CovidStratified/Data/premChn.csv', index_col=0).to_numpy(),
"gbr" : pd.read_csv('CovidStratified/Data/premGbr.csv', index_col=0).to_numpy(),
"isr" : pd.read_csv('CovidStratified/Data/premIsr.csv', index_col=0).to_numpy(),
"ita" : pd.read_csv('CovidStratified/Data/premIta.csv', index_col=0).to_numpy(),
"nld" : pd.read_csv('CovidStratified/Data/premNLD.csv', index_col=0).to_numpy(),
}
# to retrieve entries: prem["can"]

countries = list(prem)

print(cases)
print(prem["can"])


     x00  x10   x20    x30    x40    x50    x60    x70  x80plus
can   43   53   250    301    315    380    300    163      139
chn  416  549  3619   7600   8571  10008   8583   3918     1408
gbr  701  822  7299  10048  12319  15441  12081  14655    24325
isr  319  732  1865   1206   1051   1134    969    495      247
ita   63  118   511    819   1523   2480   2421   2849     2533
nld   71  214  1973   2152   2839   4719   3805   4531     6219
[[ 2.87087681  0.94357477  0.37358052  0.25317159  0.33586201  0.65584323
   1.03423059  0.92004231  0.4489512   0.29422722  0.31204529  0.25033322
   0.16140108  0.13253666  0.07109389  0.03790854]
 [ 0.86595357  4.4816395   0.88978665  0.25207481  0.15642724  0.40257766
   0.76881514  0.93100172  0.74227614  0.31622672  0.21051978  0.17382395
   0.14895517  0.11229653  0.04629893  0.03547422]
 [ 0.21203609  1.41985066  6.67547793  0.80122709  0.29643418  0.25208034
   0.46080405  0.71878219  0.96370292  0.5479558   0.28606516  0.14325162
   0.0

In [3]:
# Single Country Estimation
# the bounds
theta = np.array([1]*14)
xmin = [0] * 14
xmax = [100] * 14
my_bounds = [(low, high) for low, high in zip(xmin, xmax)]

print ("Canada:\n")
kwargs = dict(args = (prem["can"], cases_dict["can"]), method="L-BFGS-B")
print(optimize.basinhopping(CovidEM.Covid_KL_k, theta, minimizer_kwargs = kwargs))

print ("\n\nChina:\n")
kwargs = dict(args = (prem["chn"], cases_dict["chn"]), method="L-BFGS-B")
print(optimize.basinhopping(CovidEM.Covid_KL_k, theta, minimizer_kwargs = kwargs).x)

print ("\n\nBritain:\n")
kwargs = dict(args = (prem["gbr"], cases_dict["gbr"]), method="L-BFGS-B")
print(optimize.basinhopping(CovidEM.Covid_KL_k, theta, minimizer_kwargs = kwargs).x)

print ("\n\nIsreal:\n")
kwargs = dict(args = (prem["isr"], cases_dict["isr"]), method="L-BFGS-B")
print(optimize.basinhopping(CovidEM.Covid_KL_k, theta, minimizer_kwargs = kwargs).x)

print ("\n\nItaly:\n")
kwargs = dict(args = (prem["ita"], cases_dict["ita"]), method="L-BFGS-B")
print(optimize.basinhopping(CovidEM.Covid_KL_k, theta, minimizer_kwargs = kwargs).x)

print ("\n\nNetherlands:\n")
kwargs = dict(args = (prem["nld"], cases_dict["nld"]), method="L-BFGS-B")
res = optimize.basinhopping(CovidEM.Covid_KL_k, theta, minimizer_kwargs = kwargs)
print(res)
print(res.x)

Canada:



C:\Users\zakst\anaconda3\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


                        fun: 0.0030597183993240767
 lowest_optimization_result:       fun: 0.0030597183993240767
 hess_inv: <14x14 LbfgsInvHessProduct with dtype=float64>
      jac: array([-1.33798198e-03, -1.38154072e-04, -1.44317198e-04, -5.92122709e-05,
        1.62717835e-03,  1.97684318e-04, -2.03873985e-04, -1.74553689e-04,
        1.49869093e-04, -8.43362702e-05, -7.54906559e-05,  6.69360405e-05,
        8.12714484e-05, -2.76628548e-05])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 255
      nit: 14
     njev: 17
   status: 0
  success: True
        x: array([0.40042564, 0.45205768, 1.19524163, 1.20766257, 1.04347856,
       1.6196775 , 1.3289501 , 2.64072052, 0.08772386, 0.57187984,
       1.13964258, 4.86704374, 2.00320135, 3.21305911])
                    message: ['requested number of basinhopping iterations completed successfully']
      minimization_failures: 53
                       nfev: 27405
                        nit: 100
                 

In [4]:
# Cumulative Estimation

theta = np.array([1]*14)
# the bounds
#xmin = [0] * 14
#xmax = [100] * 14
#my_bounds = [(low, high) for low, high in zip(xmin, xmax)]

kwargs = dict(args = (prem, cases_dict, countries), method="L-BFGS-B")
res = optimize.basinhopping(CovidEM.Covid_KL, theta, minimizer_kwargs = kwargs)
print(res)

                        fun: 0.3270929092185833
 lowest_optimization_result:       fun: 0.3270929092185833
 hess_inv: <14x14 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.01926445, -0.00731596, -0.00068503,  0.00411053, -0.00442709,
        0.02359342, -0.00048045, -0.00348088, -0.00313445, -0.00709546,
        0.00797811,  0.00575069,  0.00395694,  0.00012018])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 135
      nit: 7
     njev: 9
   status: 0
  success: True
        x: array([0.32377104, 0.51624193, 1.33130782, 1.71238872, 1.69428806,
       2.22061728, 2.19969645, 0.7667584 , 0.42906456, 0.55629107,
       0.07050722, 1.93365643, 1.19983506, 2.26775052])
                    message: ['requested number of basinhopping iterations completed successfully']
      minimization_failures: 62
                       nfev: 22560
                        nit: 100
                       njev: 1504
                          x: array([0.32377104, 0.5162

In [5]:
# Iterative Cumulative Estimations

imp.reload(CovidEM) #applies changes made to file
np.random.seed(451)

starts = 20 #number of different starting points to test
trials = 10 # Number of trials per start point
iters = (starts * trials)
theta_iter = [[0] * 33] * iters
best_res = optimize.OptimizeResult(x = 100, fun = 100, success = False) #placeholder var to track best result

print ("Cumulative country calculation with randomized starting points:\n")
kwargs = dict(args = (prem, cases_dict, countries), method="Nelder-Mead")
for i in range(0, starts):
    theta_guess = np.random.gamma(4, 1/4, 14) # mean = 1, sd = 2
    for j in range(0, trials):
        res = optimize.basinhopping(CovidEM.Covid_KL, theta_guess, minimizer_kwargs = kwargs)
        out_start = np.insert(np.insert(theta_guess, 2,1),10,1)
        out_end = np.insert(np.insert(res.x, 2,1),10,1)
        theta_iter[((i*trials)+j)] = np.append(np.append(out_start, out_end), res.fun)
        if (res.fun < best_res.fun): 
            best_res = res
            
    print(f"{((((i+1)*trials)/iters)*100):.2f}","%, ", end = "")


print(tabulate(theta_iter))


Cumulative country calculation with randomized starting points:



C:\Users\zakst\anaconda3\lib\site-packages\scipy\optimize\optimize.py:761: RuntimeWarning: invalid value encountered in subtract
  np.max(np.abs(fsim[0] - fsim[1:])) <= fatol):


5.00 %, 10.00 %, 15.00 %, 20.00 %, 25.00 %, 30.00 %, 35.00 %, 40.00 %, 45.00 %, 50.00 %, 55.00 %, 60.00 %, 65.00 %, 70.00 %, 75.00 %, 80.00 %, 85.00 %, 90.00 %, 95.00 %, 100.00 %, --------  --------  -  --------  --------  --------  --------  --------  --------  --------  -  --------  --------  --------  --------  --------  --------  ---------  -  --------  -------  -------  -------  -------  ------------  -----------  -  -----------  -----------  ----------  --------  ----------  --------
1.28832   1.58811   1  0.304628  0.654188  0.815713  1.31789   1.11245   0.323341  0.685253  1  0.720261  1.2793    0.841494  1.13993   2.12827   0.200633  0.447374   1  1.1085    1.54623  1.5857   1.64767  1.72099   4.56383      0.523098     1  1.00927      1.42477e-05  2.05995     2.53467   3.38838     0.316811
1.28832   1.58811   1  0.304628  0.654188  0.815713  1.31789   1.11245   0.323341  0.685253  1  0.720261  1.2793    0.841494  1.13993   2.12827   0.20846   0.4792     1  1.12966   1.57719  1

In [6]:
imp.reload(CovidEM) #applies changes made to file
imp.reload(CovidCI)
np.random.seed(451)

best_est = [0.219294308, 0.423309799, 1.124597388, 1.511734653, 1.560386081,
            1.727919503, 1.734579037, 3.77747199,  0.71170154,  0.769347828,
            0.104869899, 2.090248998, 1.98923028,  3.373799478]

if(False): # Log transform parameter estimates?
    for i in range(0, len(best_est)):
        best_est[i] = math.log(best_est[i])

# Initialize variables
CovidCI.set_prem(prem)
CovidCI.set_cases(cases_dict)
CovidCI.set_countries(countries)

cis1 = CovidCI.CI_calc(best_est, CovidCI.pos_Covid_KL)
print("\n\n\nDone 1\n\n\n")
cis2 = CovidCI.CI_calc(best_res.x, CovidCI.pos_Covid_KL)
print("\n\n\nDone 2\n\n\n")

Traceback (most recent call last):
  File "C:\Users\zakst\anaconda3\lib\site-packages\ci_rvm\ci_rvm.py", line 484, in find_CI_bound
    J_ = J[considered]
IndexError: boolean index did not match indexed array along dimension 0; dimension is 1 but corresponding boolean dimension is 14


IndexError: boolean index did not match indexed array along dimension 0; dimension is 1 but corresponding boolean dimension is 14

In [ ]:
thetas = best_res.x
confint = cis2

print("Overhauled KL, Estimate")
print("non-trans: theta: ", thetas[0])
print("non-trans: CIs:", confint[0,:])
print("non-trans: theta - upper: ", thetas[0] - confint[0,0])
print("non-trans: theta - lower: ", thetas[0] - confint[0,1])
print("\n")

#for i in range(0,len(thetas)):
#    print("Parameter #", i+1)
#    print("trans: theta: ", CovidCI.param_trans(thetas)[i])
#    print("trans: CIs:", CovidCI.param_trans(confint[i,:]))
#    print("trans: theta - upper:", CovidCI.param_trans(thetas)[i] - CovidCI.param_trans(confint[i,0]))
#    print("trans: theta - lower:", CovidCI.param_trans(thetas)[i] - CovidCI.param_trans(confint[i,1]))
#    print("\n")
    

for i in range(0,len(thetas)):
    print("Parameter #", i+1)
    print("trans: theta: ", (thetas)[i])
    print("trans: CIs:", (confint[i,:]))
    print("trans: theta - upper:", (thetas)[i] - (confint[i,0]))
    print("trans: theta - lower:", (thetas)[i] - (confint[i,1]))
    print("\n")

In [ ]:
ci_header = ["Age", "Lower", "Estimate", "Upper"]

ci_res = [["s0-9","s10-19","s30-39","s40-49","s50-59","s60-69","s70-79",
           "v0-9","v10-19","v30-39","v40-49","v50-59","v60-69","v70-79"],
          cis[:,0],
          best_est,
          cis[:,1]]
print(best_est - cis[:,0])
print(best_est)
print(best_est - cis[:,1])
ci_res = np.array(ci_res).T.tolist()

print(ci_res)


In [ ]:
# Example of Confidence intervals

import numpy as np               # for numerical operations
from scipy import stats          # for stats functions
from scipy import optimize as op # to maximize the likelihood

import numdifftools as nd        # to compute gradient and Hessian numerically;
                                 # the package can be found on pypi.
                                 # Another good package for that purpose
                                 # (using automatic differentiation) is autograd

from ci_rvm import find_CI

# Define the size of the data set
n = 100

# Define the true parameters
k, p = 5, 0.1

# Generate the data set
data = np.random.negative_binomial(k, p, size=n)

# Because the parameters are constrained to the positive range and the
# interval (0, 1), respectively, we work on a transformed parameter space
# with unbounded domain.
def transform_parameters(params):
   k, p = params
   return np.exp(k), 1/(1+np.exp(-p))

# Log-Likelihood function for a negative binomial model
def logL(params):
    k, p = transform_parameters(params)
    return stats.nbinom.logpmf(data, k, p).sum()

# negative log-Likelihood function for optimization (because we use
# minimization algorithms instead of maximization algorithms)
negLogL = lambda params: -logL(params)

# Initial guess
x0 = [0, 0]

# Maximize the likelihood
result = op.minimize(negLogL, x0)
print(result.x)

# Print the result (we need to transform the parameters to the original
# parameter space to make them interpretable)
print("The estimate is: k={:5.3f}, p={:5.3f}".format(*transform_parameters(result.x)))

# Define gradient and Hessian
jac = nd.Gradient(logL)
hess = nd.Hessian(logL)

# Find confidence intervals for all parameters.
# Note: For complicated problems, it is worthwile doing this in parallel.
#       However, then we would need to encapsulate the procedure in a
#       method and define the likelihood function, gradient, and Hessian
#       on the top level of the module.
CIs = find_CI(result.x, logL, jac, hess,
              disp=True) # the disp argument lets the algorithm print
                         # status messages.
    
print(result.x)
print(CIs)
    
# CIs is a 2D numpy array with CIs[i, 0] containing the lower bound of the
# confidence interval for the i-th parameter and CIs[i, 1] containing the
# respective upper bound.

# Print the confidence intervals. Note: we need to transform the parameters
# back to the original parameter space.
original_lower = transform_parameters(CIs[:,0])
original_upper = transform_parameters(CIs[:,1])
print("Confidence interval for k: [{:5.3f}, {:5.3f}]".format(
   original_lower[0], original_upper[0]))
print("Confidence interval for p: [{:5.3f}, {:5.3f}]".format(
   original_lower[1], original_upper[1]))

In [ ]:
# Example of basin hopping

import numpy as np
from scipy import optimize

# an example function with multiple minima
def f(x): return x.dot(x) + np.sin(np.linalg.norm(x) * np.pi)

# the starting point
x0 = [10., 10.]

# the bounds
xmin = [1., 1.]
xmax = [11., 11.]

# rewrite the bounds in the way required by L-BFGS-B
bounds = [(low, high) for low, high in zip(xmin, xmax)]

# use method L-BFGS-B because the problem is smooth and bounded
minimizer_kwargs = dict(method="L-BFGS-B", bounds=bounds)
res = optimize.basinhopping(f, x0, minimizer_kwargs=minimizer_kwargs)
print(res)

